# Ciência de Dados para Negócios III - Trabalho 1 - Regressão Linear

**Aluno**: Antônio Arthur Silva de Lima

**Matrícula**: 508492

**Professor**: Victor Aguiar Evangelista de Farias

**Data**: 12/10/2025

## Implementação dos Algoritmos

In [1]:
# biblioteca
import numpy as np

### Regressão Linear Simples

#### Mínimos Quadrados

As estimativas dos parâmetros do modelo simples podem ser encontradas da seguinte maneira:

<br>

$$
\hat{\beta}_1 = \dfrac{\sum_{i=1}^{n}\left(x_i - \bar{x}\right)y_i}{\sum_{i=1}^{n}\left(x_i - \bar{x}\right)^2}
$$

<br>

$$
\hat{\beta}_0 = \bar{y} - \hat{\beta}_1\bar{x}
$$

<br>

$$
\hat{y}_i = \hat{\beta}_0 + \hat{\beta}_1x_i
$$

<br>

em que $\hat{\beta}_1 = \hat{w}_1$ e $\hat{\beta}_0 = \hat{w}_0$.

In [2]:
# função para calcular as estimativas do modelo
def minimos_quadros(X, y):
  y_bar = np.mean(y)
  x_bar = np.mean(X)

  Sxy = np.sum((X - x_bar)*y)
  Sxx = np.sum((X - x_bar)**2)

  beta1 = Sxy/Sxx
  beta0 = y_bar - beta1*x_bar

  return beta0, beta1

In [3]:
# função para fazer a predição
def predict_univariate(X, beta0, beta1):
  y_predict = beta0 + beta1*X

  return y_predict

In [4]:
# funções para quantificar a qualidade do modelo

# R2
def R2(y_true, y_predict):
  y_bar = np.mean(y_true)

  SQReg = np.sum((y_predict - y_bar)**2)
  SQTot = np.sum((y_true - y_bar)**2)

  R_2 = SQReg/SQTot

  return R_2


# MSE (Erro Quadrático Médio)
def MSE(y_true, y_predict):
  n = len(y_true)
  SQRes = np.sum((y_true - y_predict)**2)

  mse = SQRes/n

  return mse


In [5]:
# função para padronização
def standardize(v):
  new_v = (v - np.mean(v))/np.std(v)

  return new_v

#### Gradiente Descendente

As estimativas dos parâmetros no algoritmo do gradiente descendente são obtidas iterativamente da seguinte maneira:

<br>

$$
\hat{\beta}_0^{(m)} = \hat{\beta}_0^{(m-1)} + \alpha \dfrac{1}{n}\sum_{i=1}^{n}e_i
$$

<br>

$$
\hat{\beta}_1^{(m)} = \hat{\beta}_1^{(m-1)} + \alpha \dfrac{1}{n}\sum_{i=1}^{n}e_ix_i
$$

<br>

em que $\alpha$ é a taxa de aprendizado, e $e_i$ é o resíduo ordinário.

In [6]:
# função para calcular as estimativas do modelo
def gradiente_desc(X, y, alpha, n_epocas):
    beta0 = np.mean(X)
    beta1 = 0
    n = len(X)

    for i in range(n_epocas):
        y_hat = beta0 + beta1*X
        e = y - y_hat

        mse = MSE(y_true = y, y_predict = y_hat)
        r2 = R2(y_true = y, y_predict = y_hat)

        print(f'Beta0: {beta0:,.3f}')
        print(f'Beta1: {beta1:,.3f}')
        print(f'MSE: {mse:,.3f}')
        print(f'R2: {r2:.3f}\n')

        soma_e = np.sum(e)
        soma_ex = np.sum(e*X)

        beta0 = beta0 + alpha*(1/n)*soma_e
        beta1 = beta1 + alpha*(1/n)*soma_ex

    return beta0, beta1

### Regressão Linear Múltipla

Em modelos múltiplos trabalhamos com mais de uma variável regressora, e por consequência disso, torna-se necessário o uso de matrizes. Assim, o modelo fica então expresso da seguinte maneira:

<br>
$$
\mathbf{Y} = \mathbf{X}\boldsymbol{\beta} + \boldsymbol{\epsilon}
$$

<br>

em que

 - $\mathbf{Y_{n \times 1}}$ é o vetor de observações da variável resposta;

 - $\mathbf{X_{n \times p}}$ é a matriz de informação, sendo $p = k + 1$, com $k$ representando o número de variáveis regressoras do modelo (para modelos sem intercepto, $p = k$);

 - $\boldsymbol{\beta_{p \times 1}}$ é o vetor de parâmetros do modelo;

 - $\boldsymbol{\epsilon_{n \times 1}}$ é o vetor de componentes aleatórios, que supomos ser normalmente distribuído.

<br>

Podemos obter as estimativas dos parâmetros por meio da seguinte expressão:

<br>

$$
\hat{\boldsymbol{\beta}} = \left(\mathbf{X}^\top\mathbf{X}\right)^{-1}\mathbf{X}^\top \mathbf{Y}
$$

<br>

E então obtemos os valores ajustados da variável resposta:

<br>

$$
\begin{align}
\hat{\mathbf{Y}} &= \mathbf{X}\left(\mathbf{X}^\top\mathbf{X}\right)^{-1}\mathbf{X}^\top \mathbf{Y} \\
&=\mathbf{H}\mathbf{Y}
\end{align}
$$

em que $\mathbf{H} = \mathbf{X}\left(\mathbf{X}^\top\mathbf{X}\right)^{-1}\mathbf{X}^\top$ é chamada matriz de projeção


In [7]:
# função para calcular as estimativas do modelo
def metodo_analitico_multiplo(X, y):
  n = np.shape(X)[0]
  ones = np.ones(n)

  X = np.c_[ones, X]
  betas = np.linalg.inv(np.transpose(X) @ X) @ np.transpose(X) @ y

  return betas

In [8]:
# função para fazer a predição
def predict_multivariate(X, betas):
  n = np.shape(X)[0]
  ones = np.ones(n)

  X = np.c_[ones, X]
  y_hat = X @ betas

  return y_hat

## Experimento 1

### Carregamento do dado

In [9]:
df = np.loadtxt('boston_housing.csv', delimiter=',', skiprows=1) # carregando o conjunto

X = df[:, 1] # variável regressora 'LSTAT'
y = df[:, 3] # variável resposta 'MEDV'

X_padrao = standardize(X) # padronizando variável regressora
y_padrao = standardize(y) # padronizando variável resposta

### Treinamento

In [10]:
# ajuste comum
beta_0, beta_1 = minimos_quadros(X = X, y = y)

# ajuste com variáveis padronizadas
beta_0_z, beta_1_z = minimos_quadros(X = X_padrao, y = y_padrao)

# coeficientes com ajuste comum
print('Coeficientes com ajuste comum\n')
print(f'Beta0: {beta_0:,.3f}')
print(f'Beta1: {beta_1:,.3f}')

# coeficientes com ajuste de variáveis padronizadas
print('\nCoeficientes com ajuste de variáveis padronizadas\n')
print(f'Beta0: {beta_0_z:,.3f}')
print(f'Beta1: {beta_1_z:,.3f}')

Coeficientes com ajuste comum

Beta0: 684,138.493
Beta1: -17,759.048

Coeficientes com ajuste de variáveis padronizadas

Beta0: -0.000
Beta1: -0.761


### Avaliação

In [11]:
# estimativas do modelo comum
y_hat = predict_univariate(X = X, beta0 = beta_0, beta1 = beta_1)
y_hat_z = predict_univariate(X = X_padrao, beta0 = beta_0_z, beta1 = beta_1_z)

# qualidade do ajuste comum
r2 = R2(y_true = y, y_predict = y_hat)
mse = MSE(y_true = y, y_predict = y_hat)

# qualidade do ajuste com variáveis padronizadas
r2_z = R2(y_true = y_padrao, y_predict = y_hat_z)
mse_z = MSE(y_true = y_padrao, y_predict = y_hat_z)

# Qualidade do ajuste do modelo comum
print('Qualidade do ajuste do modelo comum\n')
print(f'R2: {r2:.3f}')
print(f'MSE: {mse:,.3f}')

# Qualidade do ajuste do modelo com variáveis padronizadas
print('\nQualidade do ajuste do modelo com variáveis padronizadas\n')
print(f'R2: {r2_z:.3f}')
print(f'MSE: {mse_z:,.3f}')

Qualidade do ajuste do modelo comum

R2: 0.579
MSE: 11,495,908,520.487

Qualidade do ajuste do modelo com variáveis padronizadas

R2: 0.579
MSE: 0.421


## Experimento 2

### Carregamento do dado

In [12]:
df = np.loadtxt('boston_housing.csv', delimiter=',', skiprows=1) # carregando o conjunto

X = df[:, 1] # variável regressora 'LSTAT'
y = df[:, 3] # variável resposta 'MEDV'

X_padrao = standardize(X) # padronizando variável regressora
y_padrao = standardize(y) # padronizando variável resposta

### Treinamento

In [13]:
# ajuste comum
beta_0, beta_1 = gradiente_desc(X = X, y = y, alpha = 0.005, n_epocas = 100_000)

A saída de streaming foi truncada nas últimas 5000 linhas.
Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2: 0.579

Beta0: 684,138.493
Beta1: -17,759.048
MSE: 11,495,908,520.487
R2:

In [14]:
# ajuste com variáveis padronizadas
beta_0_z, beta_1_z = gradiente_desc(X = X_padrao, y = y_padrao, alpha = 0.005, n_epocas = 100_000)

A saída de streaming foi truncada nas últimas 5000 linhas.
Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2: 0.579

Beta0: -0.000
Beta1: -0.761
MSE: 0.421
R2

### Avaliação

In [15]:
# coeficientes finais pelo ajuste comum
print(f'Coeficientes finais pelo ajuste comum\n')
print(f'Beta0: {beta_0:,.3f}')
print(f'Beta1: {beta_1:,.3f}')

# coeficientes finais pelo ajuste com variáveis transformadas
print(f'\nCoeficientes finais pelo ajuste comum\n')
print(f'Beta0: {beta_0_z:.3f}')
print(f'Beta1: {beta_1_z:.3f}')

Coeficientes finais pelo ajuste comum

Beta0: 684,138.493
Beta1: -17,759.048

Coeficientes finais pelo ajuste comum

Beta0: -0.000
Beta1: -0.761


In [16]:
# MSE e R2 finais

## ajuste comum
y_hat = predict_univariate(X = X, beta0 = beta_0, beta1= beta_1)

mse_final = MSE(y_true = y, y_predict = y_hat)
r2_final = R2(y_true = y, y_predict = y_hat)

print(f'Qualidade do ajuste para o ajuste comum\n')
print(f'MSE: {mse_final:,.3f}')
print(f'R2: {r2_final:,.3f}')

## ajuste com variáveis transformadas
y_hat_z = predict_univariate(X = X_padrao, beta0 = beta_0_z, beta1 = beta_1_z)

mse_z_final = MSE(y_true = y_padrao, y_predict = y_hat_z)
r2_z_final = R2(y_true = y_padrao, y_predict = y_hat_z)

print(f'\nQualidade do ajuste para o ajuste com variáveis transformadas\n')
print(f'MSE: {mse_z_final:,.3f}')
print(f'R2: {r2_z_final:,.3f}')

Qualidade do ajuste para o ajuste comum

MSE: 11,495,908,520.487
R2: 0.579

Qualidade do ajuste para o ajuste com variáveis transformadas

MSE: 0.421
R2: 0.579


## Experimento 3

### Carregamento do dado

In [17]:
df = np.loadtxt('boston_housing.csv', delimiter=',', skiprows=1) # carregando o conjunto

X = df[:, :3] # variáveis regressoras
y = df[:, 3] # variável resposta

X_z = np.apply_along_axis(standardize, axis = 0, arr = X) # variáveis regressoras padronizadas
y_z = standardize(y) # variável resposta padronizada

### Treinamento

In [18]:
# ajuste comum
betas = metodo_analitico_multiplo(X = X, y = y)

# ajuste com variáveis padronizadas
betas_z = metodo_analitico_multiplo(X = X_z, y = y_z)

In [19]:
# coeficientes do modelo comum
print('Coeficientes do modelo comum\n')
for i in range(len(betas)):
  print(f'Beta{i}: {betas[i]:,.3f}')

# coeficientes do modelo com variáveis transformadas
print('\nCoeficientes do modelo com variáveis transformadas\n')
for i in range(len(betas_z)):
  print(f'Beta{i}: {betas_z[i]:,.3f}')

Coeficientes do modelo comum

Beta0: 415,464.397
Beta1: 86,565.236
Beta2: -10,849.340
Beta3: -19,492.116

Coeficientes do modelo com variáveis transformadas

Beta0: 0.000
Beta1: 0.337
Beta2: -0.465
Beta3: -0.249


### Avaliação

In [20]:
# predição comum
y_hat = predict_multivariate(X = X, betas = betas)

# predição com variáveis transformadas
y_hat_z = predict_multivariate(X = X_z, betas = betas_z)

In [21]:
# MSE e R2 do modelo comum
mse = MSE(y_true = y, y_predict = y_hat)
r2 = R2(y_true = y, y_predict = y_hat)

print('Qualidade do ajuste do modelo comum\n')
print(f'MSE: {mse:,.3f}')
print(f'R2: {r2:,.3f}')

# MSE e R2 do modelo com variáveis transformadas
mse_z = MSE(y_true = y_z, y_predict = y_hat_z)
r2_z = R2(y_true = y_z, y_predict = y_hat_z)

print('\nQualidade do ajuste do modelo comum\n')
print(f'MSE: {mse_z:,.3f}')
print(f'R2: {r2_z:,.3f}')

Qualidade do ajuste do modelo comum

MSE: 7,703,545,538.871
R2: 0.718

Qualidade do ajuste do modelo comum

MSE: 0.282
R2: 0.718
